### Pandas Data Types
Accompanying the PB Python article [here](http://pbpython.com/pandas_dtypes.html)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("https://github.com/chris1610/pbpython/blob/master/data/sales_data_types.csv?raw=True")

In [ ]:
df

Use df.info and df.dtypes to look at the types that pandas automatically infers based on the data

In [ ]:
df.info()

In [ ]:
df.dtypes

Since the 2016 and 2017 columns were read in as objects, trying to add the values will result in string concatenation not numerical addition

In [ ]:
df['2016'] + df['2017']

The simplest way to to convert to a type is using astype.

We can apply it to the customer number first.

In [ ]:
df['Customer Number'].astype('int')

In [ ]:
df

The code above does not alter the original dataframe

In [ ]:
df.dtypes

Assign the new integer customer number back to the original frame and check the type

In [ ]:
df["Customer Number"] = df['Customer Number'].astype('int')
df.dtypes

In [ ]:
df

The data all looks good for the Customer Number.

If we try to convert the Jan Units column, we will get an error.

In [ ]:
df['Jan Units'].astype('int')

In a similar manner we get an error if we try to convert the sales column

In [ ]:
df['2016'].astype('float')

We can try to use astype with a bool type but that does not give expected results

In [ ]:
df['Active'].astype('bool')

In [ ]:
# astype can take a dictionary of column names and data types
df.astype({'Customer Number': 'int', 'Customer Name': 'str'}).dtypes

In order to convert the currency and percentages, we need to use custom functions

In [ ]:
def convert_currency(val):
    """
    125000.00
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','').replace('$', '')
    return float(new_val)

In [ ]:
def convert_percent(val):
    """
    Convert the percentage string to an actual floating point percent
    """
    new_val = val.replace('%', '')
    return float(new_val) / 100

Use apply to convert the 2016 and 2017 columns to floating point numbers

In [ ]:
df['2016'].apply(convert_currency)

In [ ]:
df['2017'].apply(convert_currency)

In [ ]:
df

In [ ]:
df['2016'].apply(convert_currency) + df['2017'].apply(convert_currency)

We could use a lambda function as well but it may be more difficult for new users to understand

In [ ]:
df['2016'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')

In [ ]:
# Assign the converted values back to the columns
df['2016'] = df['2016'].apply(convert_currency)
df['2017'] = df['2017'].apply(convert_currency)

In [ ]:
df

Use a lambda function to convert the percentage strings to numbers

In [ ]:
df['Percent Growth'] = df['Percent Growth'].apply(lambda x: x.replace('%', '')).astype('float') / 100

In [ ]:
df['Percent Growth'] = df['Percent Growth'].apply(convert_percent)

In [ ]:
df.dtypes

In [ ]:
# Let's look at the data so far
df

pd.to_numeric is another option for handling column conversions when invalid values are included

In [ ]:
pd.to_numeric(df['Jan Units'], errors='coerce')

In [ ]:
# Fill in the NaN with 0
pd.to_numeric(df['Jan Units'], errors='coerce').fillna(0)

Make sure to populate the original column of data

In [ ]:
df["Jan Units"] = pd.to_numeric(df['Jan Units'], errors='coerce').fillna(0)

pd.to_datetime is very useful for working with date conversions

In [ ]:
df[ ['Month', 'Day', 'Year'] ]

In [ ]:
pd.to_datetime(df[['Month', 'Day', 'Year']])

In [ ]:
df["Start_Date"] = pd.to_datetime(df[['Month', 'Day', 'Year']])

In [ ]:
# Check out the dataframe
df

Use np.where to convert the active column to a boolean

In [ ]:
df["Active"] = np.where(df["Active"] == "Y", True, False)

In [ ]:
df

In [ ]:
df.dtypes

Many of the examples shown above can be used when reading in data using dtypes or converters arguments

In [ ]:
df_2 = pd.read_csv("https://github.com/chris1610/pbpython/blob/master/data/sales_data_types.csv?raw=True", 
                   dtype={'Customer Number':'int'},
                   converters={'2016':convert_currency,
                               '2017': convert_currency,
                               'Percent Growth': convert_percent,
                               'Jan Units': lambda x: pd.to_numeric(x, errors='coerce'),
                               'Active': lambda x: np.where(x == "Y", True, False)
                              })

In [ ]:
df_2.dtypes

In [ ]:
df_2

In [ ]:
# This can not be applied at the time the data is read in
df_2["Start_Date"] = pd.to_datetime(df_2[['Month', 'Day', 'Year']])

In [ ]:
df_2